In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory




import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import xgboost as xgb

In [ ]:
#Model 1 - MLP
batch_size = 4096
hidden_units = [384, 896, 896, 394]
dropout_rates = [0.10143786981358652, 0.19720339053599725, 0.2703017847244654, 0.23148340929571917, 0.2357768967777311]
label_smoothing = 1e-2
learning_rate = 1e-3 

def create_mlp(num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate):
    
    inp = tf.keras.layers.Input(shape = (num_columns, ))
    init = tf.constant_initializer(array)
    l1 = tf.keras.layers.Dense(units=len(features),kernel_initializer=init)(inp)
    x = tf.keras.layers.BatchNormalization()(l1)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)): 
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i+1])(x)    
        
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation('sigmoid')(x)
    
    model = tf.keras.models.Model(inputs = inp, outputs = out)
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
                  loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = label_smoothing), 
                  metrics = tf.keras.metrics.AUC(name = 'AUC'), 
                 )
    
    return model

In [ ]:
#LSTM parameters
# LSTM parameters
lookback = 10
batch_size = 4096
head_hidden_units = [256]
lstm_units = [64]
tail_hidden_units = [512, 394]
dropout_rates = [0.20143786981358652, 0.19720339053599725, 0.2123435323 ,0.23148340929571917, 0.2457768967777311]


label_smoothing = 1e-2
learning_rate = 1e-3 
#Model2 - LSTM
def create_lstm(lookback, num_columns, num_labels, head_hidden_units,lstm_units,tail_hidden_units, dropout_rates, label_smoothing, learning_rate):
    
    assert (len(dropout_rates)==1 + len(head_hidden_units) + len(lstm_units)+ len(tail_hidden_units)), "number of dropout_rates is not equal to number of layers!"  
    
    inp = tf.keras.layers.Input(shape = (lookback,num_columns,))
    init = tf.constant_initializer(array)
    l1 = tf.keras.layers.Dense(units=len(features),kernel_initializer=init)(inp)
    x = tf.keras.layers.BatchNormalization()(l1)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    
    # implement hidden_layers before LSTMs
    for i in range(len(head_hidden_units)): 
        x = tf.keras.layers.Dense(head_hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i+1])(x)
    # implement LSTMs
    for i in range(len(lstm_units)):
        x = tf.compat.v1.keras.layers.CuDNNLSTM(lstm_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(dropout_rates[i+1+len(head_hidden_units)])(x) 
        
    # implement hidden_layers after LSTMs
    for i in range(len(tail_hidden_units)): 
        x = tf.keras.layers.Dense(tail_hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i+1+len(head_hidden_units)+len(lstm_units)])(x)    
        
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation('sigmoid')(x)
    
    model = tf.keras.models.Model(inputs = inp, outputs = out)
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
                  loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = label_smoothing), 
                  metrics = tf.keras.metrics.AUC(name = 'AUC'), 
                 )
    
    return model

In [ ]:
meta_model_clf = xgb.XGBClassifier(
    n_estimators=400,
    max_depth=7,
    eta=0.5, # learning_rate
    missing=None,
    random_state=42,
    #tree_method='gpu_hist',
    subsample=0.8,
    colsample_bytree=1,
    #sampling_method='gradient_based',
    #eval_metric='logloss',
    verbosity=2   # info
)

In [ ]:
import pickle
meta_model_clf = pickle.load(open("/kaggle/input/xgboost-models/pima.pickle.dat", "rb"))

In [ ]:
array = np.zeros((113,113))

In [ ]:
model_mlp = create_mlp(len(features), 1, hidden_units,dropout_rates, label_smoothing, learning_rate)
model_mlp.load_weights("/kaggle/input/testing/JSModelCV_MLP_4.hdf5")

In [ ]:


model_lstm = create_lstm(1, len(features), 1, head_hidden_units,lstm_units,tail_hidden_units, dropout_rates, label_smoothing, learning_rate) 
model_lstm.load_weights(f'/kaggle/input/testing/JSModelCV_LSTM_4.hdf5')


In [ ]:
weights = pd.read_csv('/kaggle/input/tabnet-we/Tabnet_weights.csv')
features = list(weights['Feature'])
len(features)

In [ ]:
f_mean = np.array([ 0.4424  ,  0.4636  , -0.0113  , -0.01884 , -0.0229  , -0.0372  ,
        0.0384  ,  0.01365 ,  0.2854  ,  0.231   ,  0.0883  ,  0.0631  ,
        0.1599  ,  0.1171  ,  0.2352  ,  0.2106  ,  0.1144  ,  0.1047  ,
        0.2717  ,  0.2429  ,  0.18    ,  0.1708  ,  0.2369  ,  0.2208  ,
        0.2686  ,  0.2443  ,  0.1317  ,  0.1632  ,  0.2964  ,  0.3137  ,
        0.2184  ,  0.2585  ,  0.298   ,  0.3333  ,  0.3105  ,  0.3418  ,
        0.01668 ,  0.00395 ,  0.03458 ,  0.04456 ,  0.3408  ,  0.4534  ,
        0.2683  ,  0.296   ,  0.1874  ,  0.3767  ,  0.4094  ,  0.4922  ,
        0.514   ,  0.338   ,  0.4001  , -0.012085,  0.3958  ,  0.288   ,
        0.299   ,  0.3835  ,  0.317   ,  0.316   ,  0.3276  ,  0.546   ,
        0.5483  ,  0.5146  ,  0.5137  ,  0.3953  ,  0.585   ,  0.5903  ,
        0.579   ,  0.5806  ,  0.4424  ,  0.276   ,  0.4316  ,  0.01331 ,
       -0.03613 ,  0.00651 , -0.01566 , -0.03305 , -0.06946 ,  0.01355 ,
       -0.03162 ,  0.01108 , -0.002892, -0.02783 , -0.03934 ,  0.428   ,
        0.552   ,  0.4338  ,  0.466   ,  0.5103  ,  0.4521  ,  0.2145  ,
        0.3757  ,  0.2544  ,  0.2627  ,  0.304   ,  0.2122  ,  0.4304  ,
        0.5586  ,  0.435   ,  0.4573  ,  0.503   ,  0.4568  ,  0.1692  ,
        0.371   ,  0.2654  ,  0.267   ,  0.2893  ,  0.2065  ,  0.4268  ,
        0.49    ,  0.4373  ,  0.4448  ,  0.4778  ])

In [ ]:
f_mean.shape

In [ ]:
th = 0.5 # 0.502
f = np.median



from tqdm import tqdm
import janestreet
janestreet.make_env.__called__ = False
env = janestreet.make_env()

pred_df_list = [] 

# speed up___
test_df_columns = ['weight'] + features + ['date']
index_features = [n for n, col in enumerate(test_df_columns) if col in features]

for (test_df, pred_df) in tqdm(env.iter_test()):
    
  
    if test_df['weight'].values[0] > 0:
        #test_df = test_df.fillna(0)
        test_df = test_df.loc[:,test_df_columns]
    
        x_tt = test_df[features].fillna(0).values.reshape(1,-1)
            
        pred_mlp = model_mlp(x_tt, training=False).numpy()[0][0] 
        pred_lstm = model_mlp(x_tt, training=False).numpy()[0][0] 
        
        x_tt = np.append(np.append(x_tt, pred_mlp),pred_lstm) 
        pred_xgb = meta_model_clf.predict(x_tt.reshape(1,-1)) 
    
    else:
    
        pred_df['action'].values[0] = 0
        
    env.predict(pred_df)
    pred_df_list.append(list(pred_df['action'])[0])

In [ ]:
import pandas as pd
submission = pd.read_csv('../input/jane-street-market-prediction/example_sample_submission.csv')

In [ ]:
submission['action'] = pred_df_list
submission['action'] = submission['action'].astype(int)

In [ ]:
for i in range(0,len(pred_df_list)):
    if submission['action'][i] !=0 and submission['action'][i] != 1:
        print(submission['action'][i])

In [ ]:
submission.to_csv('submission.csv' , index=False)

In [ ]:
submission.dtypes